# **LLM BRAIN**

In [28]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os
import json
from pydantic import BaseModel,Field
from typing import List
from datetime import datetime
import time

load_dotenv()

if os.environ.get('GEMINI_API_KEY'):
    print("API KEY LOADED SUCCESSFULLY")
else:
    raise AttributeError("API KEY NOT FOUND")


API KEY LOADED SUCCESSFULLY



# **PROMPTS**

In [29]:


def get_system_prompt():
    return """
You are a Senior Pinterest Creative Strategist and Visual Director.

Your task is to generate a HIGH-END, PROFESSIONAL Pinterest Pin concept
that is visually polished, scroll-stopping, and optimized for saves.

Return ONLY valid JSON that strictly matches the schema below.

{{
  "pin_intent": {{
    "primary_goal": "The core goal of the pin (e.g., Inspire, Educate, Drive Clicks)",
    "target_audience": "Specific audience persona",
    "search_intent": "Pinterest search intent (Informational, Inspirational, Transactional)"
  }},
  "visual_concept": {{
    "scene_description": "Clear description of the overall scene and composition",
    "subject_focus": "Main visual subject",
    "background_style": "Background aesthetic (e.g., Minimal, Lifestyle, Textured)",
    "lighting_style": "Lighting mood (e.g., Soft Natural, High Contrast)",
    "camera_angle": "Camera framing (e.g., Flat lay, Eye-level, Top-down)"
  }},
  "image_generation_prompt": {{
    "prompt": "Ultra-detailed prompt for Midjourney/DALL·E",
    "negative_prompt": "What to avoid visually",
    "aspect_ratio": "Pinterest-optimized ratio (e.g., 2:3)",
    "quality_tags": ["High resolution", "Editorial quality", "Pinterest aesthetic"]
  }},
  "text_overlay": {{
    "headline": "Short, bold hook text for the image",
    "supporting_text": "Optional secondary line (can be empty)",
    "font_style": "Font pairing style (e.g., Bold Serif + Clean Sans)",
    "text_alignment": "Left, Center, or Right",
    "placement": "Exact placement on image (e.g., Upper third)"
  }},
  "color_and_style": {{
    "color_palette": ["#HEX1", "#HEX2", "#HEX3"],
    "design_style": "Overall design style (e.g., Minimalist, Editorial, Cozy)",
    "emotional_vibe": "Emotional tone (e.g., Calm, Aspirational, Energetic)"
  }},
  "seo_metadata": {{
    "pin_title": "SEO-optimized Pinterest title",
    "pin_description": "Keyword-rich description with 3–5 hashtags",
    "primary_keywords": ["keyword1", "keyword2", "keyword3"]
  }}
}}

No explanations.
No markdown.
JSON output only.
"""



prompt = ChatPromptTemplate.from_messages([
    ("system", get_system_prompt()),
    ("human", "{topic}")
])

# **CLASSES FOR PYDANTIC SCHEMA**

In [30]:
class PinIntent(BaseModel):
    primary_goal: str = Field(description="Main objective of the pin")
    target_audience: str = Field(description="Specific audience persona")
    search_intent: str = Field(description="Pinterest search intent")


class VisualConcept(BaseModel):
    scene_description: str = Field(description="Overall scene and composition")
    subject_focus: str = Field(description="Main subject of the image")
    background_style: str = Field(description="Background aesthetic")
    lighting_style: str = Field(description="Lighting mood")
    camera_angle: str = Field(description="Camera framing or angle")


class ImageGenerationPrompt(BaseModel):
    prompt: str = Field(description="Highly detailed image generation prompt")
    negative_prompt: str = Field(description="Visual elements to avoid")
    aspect_ratio: str = Field(description="Pinterest-friendly aspect ratio")
    quality_tags: List[str] = Field(description="Quality and realism tags")


class TextOverlay(BaseModel):
    headline: str = Field(description="Primary hook text on the image")
    supporting_text: str = Field(description="Secondary optional text")
    font_style: str = Field(description="Font pairing or style")
    text_alignment: str = Field(description="Text alignment")
    placement: str = Field(description="Exact placement on image")


class ColorAndStyle(BaseModel):
    color_palette: List[str] = Field(description="3 hex colors")
    design_style: str = Field(description="Overall design style")
    emotional_vibe: str = Field(description="Emotional tone")


class SeoMetadata(BaseModel):
    pin_title: str = Field(description="SEO-optimized Pinterest title")
    pin_description: str = Field(description="Keyword-rich description with hashtags")
    primary_keywords: List[str] = Field(description="Main SEO keywords")


class PinterestPinStrategy(BaseModel):
    pin_intent: PinIntent
    visual_concept: VisualConcept
    image_generation_prompt: ImageGenerationPrompt
    text_overlay: TextOverlay
    color_and_style: ColorAndStyle
    seo_metadata: SeoMetadata


# **HELPER FUNCTION**

In [31]:
def log_to_daily_jsonl(data_object, folder="logs"):
    if not os.path.exists(folder):
        os.makedirs(folder)
    date_str = datetime.now().strftime("%Y-%m-%d")
    filename = os.path.join(folder, f"{date_str}_pinterest.jsonl")
    entry = data_object.dict()
    entry["logged_at"] = datetime.now().isoformat()
    with open(filename, "a", encoding="utf-8") as f:
        f.write(json.dumps(entry) + "\n")
    return filename

client = ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0.7)

def generate_trending_topic():
    response = client.invoke(
           [ {
                "role": "user",
                "content": (
                    "You are a Pinterest trend analyst. "
                    "Return ONE short, high-performing Pinterest topic. "
                    "No explanation. No formatting. Just the topic."
                )
            }
           ]
    )

    return response.content.strip()


# **OUTPUT OF THE BRAIN**

In [32]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0,response_mime_type="application/json")
structured_llm = model.with_structured_output(PinterestPinStrategy)

chain = prompt | structured_llm
USER_PROMPT = generate_trending_topic()
try:
    result = chain.invoke({'topic':USER_PROMPT})

    with open('current_strategy.json', 'w') as f:
        json.dump(result.dict(), f)
    log_file = log_to_daily_jsonl(result, folder="logs")
except Exception as e:
    print(f"❌ Error: {e}")

C:\Users\souga\AppData\Local\Temp\ipykernel_20420\1905953225.py:10: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  json.dump(result.dict(), f)
C:\Users\souga\AppData\Local\Temp\ipykernel_20420\962052498.py:6: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  entry = data_object.dict()
